# Multi-Modal Data Preparation

In [15]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
from torch.utils.data import DataLoader
import ast



In [6]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [11]:
clean_data = pd.read_csv("dataset/dataset_transformed.csv")
clean_data.head()

,tweet_text,image_path,labels,input_ids,attention_mask,transformed_image_path
0,rt xxsugvngxx i ran into this holy nigga today,dataset/images/0.jpg,"[1, 0, 0]","tensor([ 101, 19387, 22038, 6342, 2290, 160...","tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",dataset/transformed_images/0.pt
1,everybody calling you nigger now,dataset/images/1.jpg,"[1, 0, 1]","tensor([ 101, 7955, 4214, 2017, 9152, 133...","tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,...",dataset/transformed_images/1.pt
2,fuck ice white supremacist trash all of you ar...,dataset/images/2.jpg,"[5, 1, 1]","tensor([ 101, 6616, 3256, 2317, 10514, 281...","tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",dataset/transformed_images/2.pt
3,sissy faggot,dataset/images/3.jpg,"[3, 3, 0]","tensor([ 101, 24761, 6508, 6904, 13871, 41...","tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,...",dataset/transformed_images/3.pt
4,nigga what,dataset/images/4.jpg,"[4, 0, 3]","tensor([ 101, 9152, 23033, 2054, 102, ...","tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,...",dataset/transformed_images/4.pt


In [16]:

# Convert string representation of lists to actual lists
clean_data["labels"] = clean_data["labels"].apply(ast.literal_eval)

In [ ]:
# Creating the MultiModalDataset class
class MultiModalDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        
        # Text data
        text = row["tweet_text"]
        inputs = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        input_ids = inputs["input_ids"].squeeze(0)
        attention_mask = inputs["attention_mask"].squeeze(0)
        
        # Image data
        image_path = row["transformed_image_path"]
        image = torch.load(image_path)  # Load the transformed image tensor
        
        # Labels
        labels = torch.tensor(row["labels"], dtype=torch.float)
        
        return input_ids, attention_mask, image, labels





In [18]:
# Create the dataset
dataset = MultiModalDataset(clean_data, tokenizer)

In [19]:
# Create the DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Example: Iterate through the DataLoader
for batch in dataloader:
    input_ids, attention_mask, images, labels = batch
    print("Input IDs:", input_ids.shape)
    print("Attention Mask:", attention_mask.shape)
    print("Images:", images.shape)
    print("Labels:", labels.shape)
    break

Input IDs: torch.Size([32, 128])
Attention Mask: torch.Size([32, 128])
Images: torch.Size([32, 3, 224, 224])
Labels: torch.Size([32, 3])
